<a href="https://colab.research.google.com/github/angoro12/Introduccion-a-Capstone/blob/main/Semana_5_CompletoIBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Agrupación y Segmentación de Vecindarios en la Ciudad de Bogotá, Colombia



##Descripción del problema y una discusión de los antecedentes:

El objeto de análisis es buscar una zona que no cuente con restaurantes; un lugar donde se pueda posicionar un establecimiento especializado en comidas rápidas; al igual que, en alimentos de preparación mas elaborada para almunerzos y cenas.

La capital de Colombia es un epicentro gastronómico bastante interesante. Aquí convergen platos de distintas partes del mundo, los mejores chefs y restaurantes del país y del continente ofertan un sinnúmero de opciones para los ciudadanos. (https://bogota.gov.co/mi-ciudad/comida-tipica-bogotana-ademas-del-ajiaco)

##Descripción de los datos y cómo se utilizarán para resolver el problema:

Se extraen los datos de georeferenciación desde un archivo de excel llamado: georeferencia-puntual-por-localidad.xlsx; este, contiene los datos de georeferencia necesarios para utilizar el módulo de Geocoder.

Posteriormente se conforma el DataFrame result_df_filtered, el cual será utilizado para generar los mapas y las agrupaciones necesarias que permitan identificar la necesidad de tipo de negocio en los municipios.

Los municipios para el caso de Bogotá se pueden denominar como localidades, estas cumplirán la función de Bourogh. Las locaidades de Bogotá tienen varios barrios agrupados en un código postal (PostalCode), a estos grupos los denominaremos 'Neighborhoods'.



In [2]:
import pandas as pd

# Ruta del archivo en tu entorno de Google Colab
file_path = 'georeferencia-puntual-por-localidad.xlsx'

# Cargar el archivo Excel en un DataFrame
df = pd.read_excel(file_path)

# Seleccionar las columnas requeridas: PostalCode, Borough, Neighborhood
selected_columns = ['PostalCode', 'Borough', 'Neighborhood']
result_df_filtered = df[selected_columns]

# Mostrar las primeras filas del DataFrame resultante
print(result_df_filtered.head())


   PostalCode         Borough  \
0      111511  ANTONIO NARIÑO   
1      111211  BARRIOS UNIDOS   
2      111221  BARRIOS UNIDOS   
3      110711            BOSA   
4      110721            BOSA   

                                        Neighborhood  
0  Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...  
1  Alcázares Norte, Doce de Octubre, Entrerríos, ...  
2  Alcázares, Baquero, Benjamín Herrera, Colombia...  
3  Betania, Brasil, Brasilia, Cañaveralejo, Cañav...  
4  Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...  


# Función para obtener las coordenadas (latitud y longitud) de todos los vencindarios.

In [3]:
!pip install geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 2.2 MB/s eta 0:00:00


In [4]:
import geocoder

# Función para obtener las coordenadas (latitud y longitud) de una dirección utilizando el paquete de https://geocoder.readthedocs.io/index.html
def get_coordinates(postal_code):
    g = geocoder.arcgis('{}, Bogotá, Colombia'.format(postal_code))
    lat_lng = g.latlng
    return lat_lng

# Obtener las coordenadas para cada 'Neighborhood' usando 'PostalCode'
result_df_filtered['Latitude'], result_df_filtered['Longitude'] = zip(*result_df_filtered.apply(lambda row: get_coordinates(row['PostalCode']), axis=1))

result_df_filtered

<ipython-input-4-fa45cca673d1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_filtered['Latitude'], result_df_filtered['Longitude'] = zip(*result_df_filtered.apply(lambda row: get_coordinates(row['PostalCode']), axis=1))
<ipython-input-4-fa45cca673d1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_filtered['Latitude'], result_df_filtered['Longitude'] = zip(*result_df_filtered.apply(lambda row: get_coordinates(row['PostalCode']), axis=1))


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,111511,ANTONIO NARIÑO,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.587130,-74.096930
1,111211,BARRIOS UNIDOS,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",4.677829,-74.071605
2,111221,BARRIOS UNIDOS,"Alcázares, Baquero, Benjamín Herrera, Colombia...",4.662365,-74.078465
3,110711,BOSA,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",4.638680,-74.189920
4,110721,BOSA,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",4.632179,-74.201945
...,...,...,...,...,...
76,110531,USME,"Brisas del Llano, Chapinerito, El Bosque, El B...",4.485882,-74.084227
77,110541,USME,"Antonio José de Sucre, Centro Usme, Centro Usm...",4.476102,-74.106503
78,110551,USME,"Olarte, Caldera, Calderitas, Fruticas, San Benito",4.424220,-74.120717
79,110561,USME,Curubital,4.342889,-74.129096


#Mostrar la cantidad de municipios y barrios

In [5]:
#Mostrar la cantidad de municipios y barrios en Bogotá, Colombia
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(result_df_filtered['Borough'].unique()),
        result_df_filtered.shape[0]
    )
)

The dataframe has 20 boroughs and 81 neighborhoods.


# Librería geopy para obtener la latitud y la longitud de la Ciudad de Bogotá, Colombia.
## Para poder definir una instancia del geopy necesitaremos definir un user_agent. Nombraremos a nuestro agente ny_explorer, como se muestra a continuación.

In [6]:
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bogotá, Colombia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bogotá, Colombia 4.6529539, -74.0835643.


#Crear un mapa de Bogotá, Colombia utilizando los valores de latitud y longitud de este, a su vez se ubican las cordenadas de cada Borough

In [7]:
import folium # librería para graficar mapas
# crear un mapa de Bogotá, Colombia utilizando los valores de latitud y longitud
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(result_df_filtered['Latitude'], result_df_filtered['Longitude'], result_df_filtered['Borough'], result_df_filtered['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)

map_bogota

#Crear el proceso necesario para obtener los diferentes comercios de cada grupo de neighborhoods.

In [8]:
headers = {
    "accept": "application/json",
    "Authorization": "fsq307zKMyhmfUoacXJWCgPp0wMRhnWcVRRkAwgY8lD2ovQ="
}

In [9]:
Bogota_data = result_df_filtered

In [10]:
import requests
#Vamos a crear una función que repita el mismo proceso para todos los barrios Bogotá, Colombia
# Crear una lista para almacenar los datos
venues_list = []

# Iterar sobre las filas del DataFrame que contiene los barrios de Bogota
for index, row in Bogota_data.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    radius = 500
    LIMIT = 50

    url = 'https://api.foursquare.com/v3/places/search?&ll={},{}&radius={}&limit={}'.format(
        lat,
        lng,
        radius,
        LIMIT
    )
    response = requests.get(url, headers=headers).json()

    # Extraer los datos relevantes de la respuesta
    # y agregarlos a la lista
    for item in response['results']:
        venue_name = item['name']
        venue_lat = item['geocodes']['main']['latitude']
        venue_lng = item['geocodes']['main']['longitude']
        if 'categories' in item and len(item['categories']) > 0: # Se colocó así porque había categorias con registros vacíos.
                venue_category = item['categories'][0]['name']
        else:
                venue_category = None

        venues_list.append({
            'Neighborhood': row['Neighborhood'],
            'Neighborhood Latitude': lat,
            'Neighborhood Longitude': lng,
            'Venue': venue_name,
            'Venue Latitude': venue_lat,
            'Venue Longitude': venue_lng,
            'Venue Category': venue_category
        })

# Convertir la lista en un DataFrame
Bogota_data_venues = pd.DataFrame(venues_list)
#Tamaño del DataFrame resultante
print(Bogota_data_venues.shape)
# Mostrar el DataFrame resultante
Bogota_data_venues.head()

(2657, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.58713,-74.09693,Cabala Bar Rock,4.587394,-74.096888,Bar
1,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.58713,-74.09693,El Gran Caldas,4.587458,-74.096882,Latin American Restaurant
2,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.58713,-74.09693,Colegio Guillermo Leon Valencia Sede C,4.587036,-74.097151,High School
3,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.58713,-74.09693,Pescadero Benjamin Bohorquez,4.587709,-74.097323,Seafood Restaurant
4,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.58713,-74.09693,Pescaderia Brisas Del Mar,4.587458,-74.096882,Restaurant


#Sitios entregados por cada grupo de neighbohoods

In [11]:
#Sitios entregados por cada grupo de neighbohoods
Bogota_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Abraham Lincoln, Area Artilleria, Parque El Tunal, San Benito, San Carlos, Tunal Oriental, Tunjuelito",50,50,50,50,50,45
"Acacias Usaquen, Bella Suiza, Bosque de Pinos I, Cedritos, Cedro Narvaez, Country Club, El Contador, Ginebra, La Calleja, La Carolina, Lisboa, Los Cedros, Los Cedros Oriental, Páramo Urbano I, San Gabriel Norte",50,50,50,50,50,49
"Acevedo Tejada, Campo Eucarístico, Campín Occidental, Centro Administrativo Occidental, Centro Nariño, Ciudad Salitre Nor Oriental, Ciudad Salitre Sur Oriental, Ciudad Universitaria, El Recuerdo, El Salitre, Gran América, La Esmeralda, Nicolás de Federman, Pablo VI",50,50,50,50,50,47
"Aeropuerto El Dorado, Atahualpa, Brisas Aldea Fontibón, Charco Rural, El Refugio, Ferrocaja Fontibón, La Cabaña Fontibón, Las Navetas, Pueblo Viejo, Puerta de Teja, San José de Fontibón, Versalles Fontibón",50,50,50,50,50,48
"Alamos, El Cedro, El Madrigal, El Muelle, Garcés Navas, Garcés Navas Oriental, Los Ángeles, San Antonio Engativá, Santa Mónica, Villa Amalia, Villa del Mar, Villa Sagrario",50,50,50,50,50,47
...,...,...,...,...,...,...
"Escuela de Caballería I, Escuela de Caballería II, Escuela de Infantería, Molinos Norte, Páramo, Páramo Urbano II, Páramo Urbano IV, Rincón del Chicó, San Patricio, Santa Ana, Santa Ana Occidental, Santa Bibiana, Santa Bárbara Central, Santa Bárbara Occid",50,50,50,50,50,49
"Granja de Santa Sofía, Granja San Pablo, Gustavo Restrepo, Hospital San Carlos, La Resurrección, La Resurrección I, Marco Fidel Suárez, San Jorge Sur, San José Sur, San Luis, Sosiego Sur",50,50,50,50,50,43
"La Catedral, La Concordia, Las Aguas, Centro Administrativo, Egipto, Belén , Nueva Santa Fe, Santa Bárbara",50,50,50,50,50,50


In [12]:
#Categorías únicas que se pueden conservar de todos los sitios regresados
print('There are {} uniques categories.'.format(len(Bogota_data_venues['Venue Category'].unique())))

There are 337 uniques categories.


#Clasificar en cada grupo de neighborhoods si tiene o no una categoria de sitio.

In [13]:
# codificación armar columnas con todos los valores únicos de la columna 'Venue Category'
Bogota_onehot = pd.get_dummies(Bogota_data_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir  al DataFrame 'Bogota_onehot' la columna 'Neighborhood' de regreso del dataframe 'Bogota_data_venues'
Bogota_onehot['Neighborhood'] = Bogota_data_venues['Neighborhood']

# mover la columna 'Neighborhood' a la primera columna
fixed_columns = [Bogota_onehot.columns[-1]] + list(Bogota_onehot.columns[:-1])
Bogota_onehot =Bogota_onehot[fixed_columns]

Bogota_onehot.head()

,Neighborhood,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Agriculture and Forestry Service,Alternative Medicine Clinic,Ambulance Service,American Restaurant,Amusement Park,Animal Shelter,...,Veterinarian,Video Games Store,Video Store,Vintage and Thrift Store,Water Treatment Service,Website Designer,Wholesaler,Winery,Wings Joint,Women's Store
0,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Aquí se agrupa por barrio y despues de una suma se coloca la cantidad de establecimientos por categoría en casa barrio.
#Si se utiliza count() cuenta ceros '0' y unos '1', por igual como si hubiera siempre existencia de algún tipo de categoria.
df=Bogota_onehot.groupby('Neighborhood').sum()
print(df.shape)# tamaño del nuevo DataFrame
df

(62, 336)


,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Agriculture and Forestry Service,Alternative Medicine Clinic,Ambulance Service,American Restaurant,Amusement Park,Animal Shelter,Arcade,...,Veterinarian,Video Games Store,Video Store,Vintage and Thrift Store,Water Treatment Service,Website Designer,Wholesaler,Winery,Wings Joint,Women's Store
Neighborhood,,,,,,,,,,,,,,,,,,,,,
"Abraham Lincoln, Area Artilleria, Parque El Tunal, San Benito, San Carlos, Tunal Oriental, Tunjuelito",0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
"Acacias Usaquen, Bella Suiza, Bosque de Pinos I, Cedritos, Cedro Narvaez, Country Club, El Contador, Ginebra, La Calleja, La Carolina, Lisboa, Los Cedros, Los Cedros Oriental, Páramo Urbano I, San Gabriel Norte",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Acevedo Tejada, Campo Eucarístico, Campín Occidental, Centro Administrativo Occidental, Centro Nariño, Ciudad Salitre Nor Oriental, Ciudad Salitre Sur Oriental, Ciudad Universitaria, El Recuerdo, El Salitre, Gran América, La Esmeralda, Nicolás de Federman, Pablo VI",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Aeropuerto El Dorado, Atahualpa, Brisas Aldea Fontibón, Charco Rural, El Refugio, Ferrocaja Fontibón, La Cabaña Fontibón, Las Navetas, Pueblo Viejo, Puerta de Teja, San José de Fontibón, Versalles Fontibón",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Alamos, El Cedro, El Madrigal, El Muelle, Garcés Navas, Garcés Navas Oriental, Los Ángeles, San Antonio Engativá, Santa Mónica, Villa Amalia, Villa del Mar, Villa Sagrario",0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Escuela de Caballería I, Escuela de Caballería II, Escuela de Infantería, Molinos Norte, Páramo, Páramo Urbano II, Páramo Urbano IV, Rincón del Chicó, San Patricio, Santa Ana, Santa Ana Occidental, Santa Bibiana, Santa Bárbara Central, Santa Bárbara Occid",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Granja de Santa Sofía, Granja San Pablo, Gustavo Restrepo, Hospital San Carlos, La Resurrección, La Resurrección I, Marco Fidel Suárez, San Jorge Sur, San José Sur, San Luis, Sosiego Sur",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
"La Catedral, La Concordia, Las Aguas, Centro Administrativo, Egipto, Belén , Nueva Santa Fe, Santa Bárbara",0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Agrupemos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría
# En realidad lo que hace es determinar el porcentaje que representa cada valor en la fila de tal manera que sumado horizontalmente resulta en 1,
# pero finalmente todos los uno '1' sumados por columna resultan en la cantidad de muestras que participan. Asi que esto no es un promedio sino una frecuencia relativa por fila.
# o un % del total por fila.
Bogota_grouped = df.groupby('Neighborhood').mean().reset_index()
Bogota_grouped

,Neighborhood,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Agriculture and Forestry Service,Alternative Medicine Clinic,Ambulance Service,American Restaurant,Amusement Park,Animal Shelter,...,Veterinarian,Video Games Store,Video Store,Vintage and Thrift Store,Water Treatment Service,Website Designer,Wholesaler,Winery,Wings Joint,Women's Store
0,"Abraham Lincoln, Area Artilleria, Parque El Tu...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Acacias Usaquen, Bella Suiza, Bosque de Pinos ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Acevedo Tejada, Campo Eucarístico, Campín Occi...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Aeropuerto El Dorado, Atahualpa, Brisas Aldea ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alamos, El Cedro, El Madrigal, El Muelle, Garc...",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,"Escuela de Caballería I, Escuela de Caballería...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,"Granja de Santa Sofía, Granja San Pablo, Gusta...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
59,"La Catedral, La Concordia, Las Aguas, Centro A...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,"La Esperanza, Páramo, Páramo I, Páramo II, Pár...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Descarga el excel del % del total que se aprecia horizontalmente, en otras palabras frecuencia relativa de cada categoria por barrio.
import pandas as pd
from google.colab import files

# Exportar el dataframe a un archivo Excel
Bogota_grouped.to_excel('Bogota_grouped.xlsx', index=False)

# Descargar el archivo Excel
files.download('Bogota_grouped.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
Bogota_grouped.shape #tamaño del nuevo Dataframe

(62, 337)

In [18]:
#Pongamos eso en el dataframe
#Primero escribamos una función para ordenar los sitios en orden descendente.
def return_most_common_venues(row, num_top_venues): # 'row'representa una fila del DataFrame que contiene todas las categorías de lugares frecuentes en un vecindario específico.
    row_categories = row.iloc[1:] #Genera un nuevo DataFrame 'row_categories' que excluye la primera columna [0] que contiene el nombre del vecindario o primera columna; llama la [1:] uno en adelante.
    row_categories_sorted = row_categories.sort_values(ascending=False)# Ordena los valores de las categorías en orden descendente por fila

    return row_categories_sorted.index.values[0:num_top_venues] #Regresa 10 columnas únicamente; mismas que ya contienen desde los mayores valores fila a fila.

In [19]:
import numpy as np # librería para manejar datos vectorizados


#Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada grupo de barrios.
# CREAR LOS NOMBRES DE LAS COLUMNAS CON UN  BUCLE FOR  EN FORMA DE LISTA PARA CONVERTIRLO EN UN DATAFRAME
#LLAMADO 'neighborhoods_venues_sorted':

num_top_venues = 10  #Esto indica el número de columnas que se van a crear y adicionar en la lista 'columns' para el nuevo DataFrame.
indicators = ['st', 'nd', 'rd'] # Esta variable crea una lista de sufijos para los números ordinales 1, 2 ,3.

columns = ['Neighborhood']# Se crea una lista inicial denominada 'columns' con una primera columna que se llamará 'Neighborhood'

for ind in np.arange(num_top_venues): # Esto crea un bucle para generar los encabezados de las columnas de los lugares más comunes (por ejemplo, "1st Most Common Venue", "2nd Most Common Venue"

    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    #Entre tanto haya valores en la lista 'indicators' posiciones [0],[1] y [2] que son los mismos 3 bucles de "ind" se conformaran los nombres de las columnas así.
    except:
    #Cuando se pasa a la siguiente posición de la lista 'indicators' y no hay mas valores solo se agrega el número al nombre que ya viente con 'th'
        columns.append('{}th Most Common Venue'.format(ind+1))
# crear un nuevo dataframe vacio con la lista de columnas creadas anteriormente:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
 # Se asignan o traspasan los valores del Dataframe Bogota_grouped['Neighborhood'] a la columna ['Neighborhood'] del nuevo DataFrame vacío neighborhoods_venues_sorted.
neighborhoods_venues_sorted['Neighborhood'] = Bogota_grouped['Neighborhood']
# El Data Frame: manhattan_grouped representa la frecuencia relativa de categoria por barrio



# CREAR UNA ITERACIÓN DE TAL MANERA QUE RESULTA LAS 10 CATEGORIAS DE MAYOR FRECUENCIA POR BARRIO:

for ind in np.arange(Bogota_grouped.shape[0]): # 'Bogota_grouped.shape[0]' arroja en una lista la enumeración de filas en df 'Bogota_grouped'.
# Este bucle for itera a través de las filas del DataFrame Bogota_grouped utilizando np.arange(Bogota_grouped.shape[0]).
#En cada iteración, se llama a la función 'return_most_common_venues' para obtener los lugares más comunes en el vecindario actual
#y se asignan a las columnas correspondientes en neighborhoods_venues_sorted.



    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bogota_grouped.iloc[ind, :], num_top_venues)
# Del lado derecho la función 'return_most_common_venues' se alimenta del
# dataframe 'Bogota_grouped' que representa la frecuencia relativa de categoria por barrio
# según el barrio que representa cada índice, nos trae las 10 columnas que identifican las categorias mas frecuentes en cada barrio,
#por acción de la función sin la columna 'neighborhood'

#De lado izquierdo toma el resultado del lado derecho y a su vez según el barrio que representa cada índice; accede a las filas y columnas especificadas
# en el DataFrame neighborhoods_venues_sorted. Utiliza iloc para indexar por posición, donde ind es el índice actual del bucle y
# 1: indica todas las columnas a partir de la columna con índice 1 (excluyendo la columna con índice 0) que es 'neighborhood', así solo permite asociar
#solamente con  las restantes 10 columnas que no tienen valores.

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abraham Lincoln, Area Artilleria, Parque El Tu...",Chemicals and Gasses Manufacturer,Education,Sporting Goods Retail,Fast Food Restaurant,Business and Professional Services,Church,Leather Supplier,Furniture and Home Store,General Surgeon,Miscellaneous Store
1,"Acacias Usaquen, Bella Suiza, Bosque de Pinos ...",Pizzeria,Burger Joint,Fast Food Restaurant,Bakery,Doctor's Office,Sports and Recreation,Café,Office Building,Big Box Store,Swimming Pool
2,"Acevedo Tejada, Campo Eucarístico, Campín Occi...",Government Building,Restaurant,Seafood Restaurant,Burger Joint,BBQ Joint,Community and Government,Office Building,Military,Deli,Courthouse
3,"Aeropuerto El Dorado, Atahualpa, Brisas Aldea ...",Office Building,Donut Shop,Post Office,Deli,"Cafe, Coffee, and Tea House","Shipping, Freight, and Material Transportation...",Repair Service,Fast Food Restaurant,Brewery,Restaurant
4,"Alamos, El Cedro, El Madrigal, El Muelle, Garc...",Fast Food Restaurant,Burger Joint,Furniture and Home Store,Medical Center,Mexican Restaurant,Toy Store,Soccer Field,Bakery,Construction,Department Store


#Crear Agrupaciones

In [20]:
##### importar k-means desde la fase de agrupación####
### Se debe crear agrupamiento con Bogota_grouped que la que cuenta con la codificación numérica.
from sklearn.cluster import KMeans
#Barrios Agrupados
#Ejecutemos k-means para agrupar los barrios en 7 agrupaciones.
# establecer el número de agrupaciones
kclusters = 7

#Bogota_grouped_clustering, es una copia de Bogota_grouped sin la columna 'Neighborhood'.
Bogota_grouped_clustering = Bogota_grouped.drop('Neighborhood', 1)

# Se ejecuta el algoritmo de K-means y mediante el método fit() es aplicado a Bogota_grouped_clustering.
#Esto significa que el algoritmo de K-means analizará los datos y encontrará los centroides óptimos para cada agrupación.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bogota_grouped_clustering)

# Este código muestra en una lista las etiquetas asignadas a cada a cada fila del dataframe;
#Las etiquetas indican a  que agrupación de las 5 pertenece cada fila. En este caso solo se escoge las primeras 10 filas.
kmeans.labels_[0:10]

<ipython-input-20-901d017f4121>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Bogota_grouped_clustering = Bogota_grouped.drop('Neighborhood', 1)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([4, 3, 3, 3, 3, 2, 1, 1, 6, 3], dtype=int32)

In [21]:
kmeans

KMeans(n_clusters=7, random_state=0)

#Generar un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.

In [22]:
# añadir etiquetas
#Bogota_grouped representa un dataframe de frecuencia relativa por fila de las 345 categorias de lugares de los 39 barrios.

#El DataFrame 'neighborhoods_venues_sorted' toma del dataframe 'Bogota_grouped' los datos y muestra las 10 categorias de sitios mas presentes en
#los 39 barrios ['neighborhood'] no en valor de frecuencia numérica sino con el nombre de cada categoria ocupando una posición horizontal por barrio,
#fila a fila.

#El algoritmo Kmeans toma nuevamente el dataframe 'Bogota_grouped' convertido en el dataframe 'Bogota_grouped_clustering' sin la columna
#'neighborhood' y con la frecuencia relativa por fila de las 345 categorias de lugares de los 39 barrios le asigna a cada fila que representa
#un barrio una de 5 etiquetas que simulan clasificación o agrupación.

#Se inserta una nueva columna llamada 'Cluster Labels' en el DataFrame neighborhoods_venues_sorted en la posición 0 (es decir, al principio del
#DataFrame). El contenido de esta columna son las etiquetas de las agrupaciones generadas por el algoritmo de K-means, que se obtienen de
#kmeans.labels_. Esto significa que cada barrio del DataFrame neighborhoods_venues_sorted ahora tiene una etiqueta de agrupación correspondiente.

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#Se crea una copia del DataFrame 'Bogota_data' llamada Bogota_merged'. Esto se realiza para mantener intacto el DataFrame original y
#trabajar con una copia. El DataFrame 'Bogota' contiene los 39 grupos de barrios; columnas Borough -	Neighborhood-Latitude-	Longitude.

Bogota_merged =Bogota_data

# Se realiza una unión (join) entre los DataFrames Bogota_merged y neighborhoods_venues_sorted utilizando la columna 'Neighborhood' como clave
#de unión. Cada fila en Bogota_merged que tenga un vecindario coincidente en neighborhoods_venues_sorted se fusionará y tendrá la etiqueta de
#agrupación correspondiente.

Bogota_merged = Bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Bogota_merged.head() # revisar el resultado donde se unen las columnas de los dos Dataframe.

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,111511,ANTONIO NARIÑO,"Policarpa, Caracas, Ciudad Berna, Ciudad Jardí...",4.587130,-74.096930,3.0,Restaurant,Education,Pizzeria,Bar,Seafood Restaurant,Café,Soccer Field,Business and Professional Services,Church,Construction
1,111211,BARRIOS UNIDOS,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",4.677829,-74.071605,1.0,Bakery,Restaurant,Hardware Store,Stationery Store,Mexican Restaurant,BBQ Joint,Frozen Yogurt Shop,Motorcycle Dealership,Furniture and Home Store,Gastropub
2,111221,BARRIOS UNIDOS,"Alcázares, Baquero, Benjamín Herrera, Colombia...",4.662365,-74.078465,1.0,Hospital,Church,Bakery,Motorcycle Dealership,Office Building,Ice Cream Parlor,Retail,Bicycle Store,Construction,Cajun and Creole Restaurant
3,110711,BOSA,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",4.638680,-74.189920,3.0,Clothing Store,Drugstore,Shopping Mall,Ice Cream Parlor,Health and Medicine,Burger Joint,Chinese Restaurant,Baby Store,Medical Center,Health Food Store
4,110721,BOSA,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",4.632179,-74.201945,3.0,Housing Development,Bar,Construction Supplies Store,Perfume Store,Business and Professional Services,Flower Store,Shopping Mall,Shoe Store,Retail,Rehabilitation Center


In [23]:
nan_values = Bogota_merged['Cluster Labels'].isna().sum()
print("Número de valores NaN:", nan_values)


Número de valores NaN: 19


In [24]:
import numpy as np

# Reemplazar NaN con un valor predeterminado, por ejemplo, -1
Bogota_merged['Cluster Labels'].fillna(-1, inplace=True)

# Convertir la columna 'Cluster Labels' a tipo entero
Bogota_merged['Cluster Labels'] = Bogota_merged['Cluster Labels'].astype(int)



In [25]:
#Descarga el excel del % del total que se aprecia horizontalmente, en otras palabras frecuencia relativa de cada categoria por barrio.
import pandas as pd
from google.colab import files

# Exportar el dataframe a un archivo Excel
Bogota_merged.to_excel('Bogota_merged.xlsx', index=False)

# Descargar el archivo Excel
files.download('Bogota_merged.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Graficar las agrupaciones.

In [26]:
# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

#Finalmente visualicemos las agrupaciones resultantes
# Se crea un objeto de mapa llamado map_clusters utilizando la función folium.Map(). Se especifica la ubicación del mapa
#con las variables de las coordenadas [latitude, longitude] y se establece el nivel de zoom inicial en 11.
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones:
x = np.arange(kclusters) #Se crea una lista x que contiene valores desde 0 hasta kclusters (el número de agrupaciones que son 5) [0,1,2,3,4]

ys = [i + x + (i*x)**2 for i in range(kclusters)]#  Luego, se genera una lista de listas llamada ys, donde cada sublista se construye sumando
# cada valor asociado al índice 'i'  del 'bucle for' a la lista completa  'x'  mas (i*la lista completa x) al cuadrado.
#Estos cálculos se realizan para obtener una distribución de colores única para cada agrupación. Por tanto cada sublista contiene como resultado
# un número entero.
#Este es el ejemplo de como se opera con listas:
#[array([0, 1, 2, 3, 4]),
 #array([ 1,  3,  7, 13, 21]),.....[1]+ [0,1,2,3,4]+ (1* [0,1,2,3,4])^2 = [1,2,3,4,5]+[0,1,2,3,4]^2= [1,2,3,4,5]+[0,1,4,9,16]=[1,3,7,13,21]
 #array([ 2,  7, 20, 41, 70]),.....[2]+ [0,1,2,3,4]+ (2* [0,1,2,3,4])^2 = [2,3,4,5,6]+[0,2,4,6,8]^2= [2,3,4,5,6]+[0,4,16,36,64]=[2,7,20,41,70]
 #array([  3,  13,  41,  87, 151]),
 #array([  4,  21,  70, 151, 264])]


colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]




# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(Bogota_merged['Latitude'], Bogota_merged['Longitude'], Bogota_merged['Neighborhood'], Bogota_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

#Examinar Agrupaciones

In [27]:
#Examinar Agrupaciones Zona Comercial
#Ahora puede examinar cada agrupación y determinar las categorias del sitio que distingue a cada agrupación. En base a las categorias definidas usted puede asignar un nombre a cada agrupación.
#Agrupación -1. Comercial
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 1, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,BARRIOS UNIDOS,1,Bakery,Restaurant,Hardware Store,Stationery Store,Mexican Restaurant,BBQ Joint,Frozen Yogurt Shop,Motorcycle Dealership,Furniture and Home Store,Gastropub
2,BARRIOS UNIDOS,1,Hospital,Church,Bakery,Motorcycle Dealership,Office Building,Ice Cream Parlor,Retail,Bicycle Store,Construction,Cajun and Creole Restaurant
10,CIUDAD BOLÍVAR,1,Education,Restaurant,Construction,Bakery,Professional Cleaning Service,South American Restaurant,Dentist,Church,Retail,Fast Food Restaurant
19,ENGATIVÁ,1,Bakery,Print Store,BBQ Joint,Construction,Retail,Construction Supplies Store,Medical Center,Medical Lab,Health and Medicine,Nursery School
53,SUBA,1,Nursery School,Education,Print Store,Hardware Store,Metals Supplier,Security and Safety,Construction,Restaurant,Drugstore,Community and Government


In [28]:
#Examinar Agrupaciones Zona Comercial
#Ahora puede examinar cada agrupación y determinar las categorias del sitio que distingue a cada agrupación. En base a las categorias definidas usted puede asignar un nombre a cada agrupación.
#Agrupación 0. Comercial
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 0, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,LOS MÁRTIRES,0,Shopping Mall,Restaurant,BBQ Joint,Women's Store,Sporting Goods Retail,Bar,Paintball Field,Clothing Store,Telecommunication Service,Grocery Store


In [29]:
#Agrupación 1.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 1, Bogota_merged.columns[[1] + list(range(10, Bogota_merged.shape[1]))]]

,Borough,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,BARRIOS UNIDOS,Mexican Restaurant,BBQ Joint,Frozen Yogurt Shop,Motorcycle Dealership,Furniture and Home Store,Gastropub
2,BARRIOS UNIDOS,Office Building,Ice Cream Parlor,Retail,Bicycle Store,Construction,Cajun and Creole Restaurant
10,CIUDAD BOLÍVAR,Professional Cleaning Service,South American Restaurant,Dentist,Church,Retail,Fast Food Restaurant
19,ENGATIVÁ,Retail,Construction Supplies Store,Medical Center,Medical Lab,Health and Medicine,Nursery School
53,SUBA,Metals Supplier,Security and Safety,Construction,Restaurant,Drugstore,Community and Government


In [30]:
#Agrupación 2.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 2, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,PUENTE ARANDA,2,Clothing Store,Burger Joint,Housing Development,Pizzeria,BBQ Joint,Church,Locksmith,Drugstore,Medical Supply Store,Fried Chicken Joint
41,RAFAEL URIBE URIBE,2,Drugstore,Construction,High School,Shoe Store,Business and Professional Services,Veterinarian,Stationery Store,"Heating, Ventilating and Air Conditioning Cont...",Park,Print Store
50,SANTA FE,2,Business and Professional Services,Church,Telecommunication Service,Construction Supplies Store,Event Service,Restaurant,Furniture and Home Store,Construction,Industrial Equipment Supplier,Nursing Home
56,SUBA,2,Pizzeria,BBQ Joint,Clothing Store,Restaurant,Housing Development,Retail,Fast Food Restaurant,Office Building,Bakery,Café
67,TUNJUELITO,2,Construction,Church,Refrigeration and Ice Supplier,Hardware Store,Office Building,General Contractor,Painter,Restaurant,Sports and Recreation,Brewery
72,USAQUÉN,2,Furniture and Home Store,Clothing Store,Construction Supplies Store,Education,Hardware Store,Engineer,Drugstore,Event Service,Business and Professional Services,Farm


In [31]:
#Agrupación 3.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 3, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANTONIO NARIÑO,3,Restaurant,Education,Pizzeria,Bar,Seafood Restaurant,Café,Soccer Field,Business and Professional Services,Church,Construction
3,BOSA,3,Clothing Store,Drugstore,Shopping Mall,Ice Cream Parlor,Health and Medicine,Burger Joint,Chinese Restaurant,Baby Store,Medical Center,Health Food Store
4,BOSA,3,Housing Development,Bar,Construction Supplies Store,Perfume Store,Business and Professional Services,Flower Store,Shopping Mall,Shoe Store,Retail,Rehabilitation Center
7,CHAPINERO,3,Diner,Argentinian Restaurant,Tennis Court,Mountain,Computer Repair Service,Shopping Mall,Shoe Store,Furniture and Home Store,Ophthalmologist,Office Supply Store
8,CHAPINERO,3,"Heating, Ventilating and Air Conditioning Cont...",Dive Spot,Electrical Equipment Supplier,Law Enforcement and Public Safety,Pawn Shop,Print Store,Technology Business,Nursery School,Organic Grocery,Ophthalmologist
9,CHAPINERO,3,Bar,Burger Joint,Food Court,Office Building,Park,Mexican Restaurant,Seafood Restaurant,Arts and Entertainment,Coffee Shop,French Restaurant
14,CIUDAD BOLÍVAR,3,Nursery School,Engineer,Jewelry Store,Arts and Crafts Store,Beer Garden,Church,Organization,Organic Grocery,Ophthalmologist,Office Supply Store
18,ENGATIVÁ,3,Pizzeria,BBQ Joint,Ice Cream Parlor,Steakhouse,Bar,Grocery Store,Clothing Store,Jewelry Store,Bowling Alley,Pub
21,ENGATIVÁ,3,Fast Food Restaurant,Burger Joint,Furniture and Home Store,Medical Center,Mexican Restaurant,Toy Store,Soccer Field,Bakery,Construction,Department Store
22,ENGATIVÁ,3,Bar,Business and Professional Services,Dentist,Deli,Metals Supplier,Medical Center,Park,Furniture and Home Store,Seafood Restaurant,Government Building


In [32]:
#Agrupación 4.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 4, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,BOSA,4,Education,Bar,Church,Restaurant,Advertising Agency,Pizzeria,Coffee Shop,Construction,Historic and Protected Site,Bakery
12,CIUDAD BOLÍVAR,4,Education,Church,Restaurant,Grocery Store,Drugstore,Construction,Health and Medicine,Business and Professional Services,Stationery Store,Furniture and Home Store
42,RAFAEL URIBE URIBE,4,Education,Grocery Store,Plastics Supplier,Business and Professional Services,Drugstore,Hospital,Library,Accounting and Bookkeeping Service,Comfort Food Restaurant,Financial Service
43,RAFAEL URIBE URIBE,4,Education,Restaurant,Business and Professional Services,Church,Furniture and Home Store,Grocery Store,Motorcycle Dealership,Fried Chicken Joint,Retail,Electronics Store
44,RAFAEL URIBE URIBE,4,Church,Education,Business and Professional Services,Construction Supplies Store,Print Store,Pest Control Service,Drugstore,Furniture and Home Store,Organization,Park
46,SAN CRISTOBAL,4,Education,Hardware Store,Drugstore,Church,Business and Professional Services,Medical Center,Grocery Store,Metals Supplier,Carpet and Flooring Contractor,Sporting Goods Retail
47,SAN CRISTOBAL,4,Education,Hospital,Grocery Store,Business and Professional Services,Bar,Restaurant,Pizzeria,Government Building,Clothing Store,Hardware Store
68,TUNJUELITO,4,Chemicals and Gasses Manufacturer,Education,Sporting Goods Retail,Fast Food Restaurant,Business and Professional Services,Church,Leather Supplier,Furniture and Home Store,General Surgeon,Miscellaneous Store


In [33]:
#Agrupación 5.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 5, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,CIUDAD BOLÍVAR,5,Chemicals and Gasses Manufacturer,Business and Professional Services,Grocery Store,Construction,Church,Bar,Park,Furniture and Home Store,Supplement Store,Print Store
20,ENGATIVÁ,5,Construction,Grocery Store,Painter,Renewable Energy Service,Plaza,Furniture and Home Store,Metals Supplier,Retail,Hardware Store,Clothing Store
32,KENNEDY,5,Sports and Recreation,Business and Professional Services,High School,Construction,Education,Grocery Store,Furniture and Home Store,Farmers Market,Soccer Field,Health Food Store
35,KENNEDY,5,Education,Security and Safety,Furniture and Home Store,Construction,Organization,Business and Professional Services,Bar,General Contractor,Restaurant,Metals Supplier
55,SUBA,5,Construction,Business and Professional Services,Housing Development,Grocery Store,Furniture and Home Store,Gift Store,Swimming Pool,Organization,Education,Government Building


In [34]:
#Agrupación 6.
Bogota_merged.loc[Bogota_merged['Cluster Labels'] == 6, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,BOSA,6,Bar,Clothing Store,Restaurant,Business and Professional Services,Music Venue,Dessert Shop,Engineer,Furniture and Home Store,Pub,Bakery
36,LA CANDELARIA,6,Bar,BBQ Joint,Falafel Restaurant,Cocktail Bar,Bookstore,Caribbean Restaurant,Library,Restaurant,Pub,Snack Place
40,PUENTE ARANDA,6,Bar,Restaurant,Pizzeria,BBQ Joint,Juice Bar,Construction,Carpet and Flooring Contractor,New American Restaurant,Taco Restaurant,Grocery Store
65,TEUSAQUILLO,6,Bar,Restaurant,Café,Burger Joint,Theater,Medical Center,Tea Room,Drugstore,Peruvian Restaurant,Bookstore
